In [4]:
import keras
from keras.layers import Conv3D,Conv2D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.layers import Add, Activation, AveragePooling2D, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras import utils
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from scipy.io import  loadmat

In [5]:
X = loadmat('/content/PaviaU.mat')['paviaU']
y = loadmat('/content/PaviaU_gt.mat')['paviaU_gt']

In [6]:
X.shape, y.shape

((610, 340, 103), (610, 340))

In [7]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX

In [8]:
X = applyPCA(X,numComponents=9)
X.shape

(610, 340, 9)

In [9]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [10]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize ) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin+ 1, c - margin:c + margin+1]
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [11]:
X, y = createImageCubes(X, y, windowSize=25)

X.shape, y.shape

((42776, 25, 25, 9), (42776,))

In [12]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.1)
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((38498, 25, 25, 9), (4278, 25, 25, 9), (38498,), (4278,))

In [13]:
ytrain = utils.to_categorical(ytrain)
ytrain.shape

(38498, 9)

In [14]:
def identity_block(X, f, filters, stage, block):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Add shortcut value to main path
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

In [15]:
def convolutional_block(X, f, filters, stage, block, s = 2):

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

In [16]:
def ResNet50(input_shape = (25, 25, 9,1), classes = 9):
    ## input layer
    input_layer = Input(input_shape)

    ## convolutional layers
    conv_layer1 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(input_layer)
    conv3d_shape = conv_layer1.shape.as_list()
    conv_layer2 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer1)
    conv_layer3 = Conv2D(filters=64, kernel_size=(5,5), activation='relu')(conv_layer2)
    bn_conv1 = BatchNormalization(axis = 3)(conv_layer3)
    bn_conv2 = Activation('relu')(bn_conv1)
    pooling_layer1 = MaxPooling2D((3, 3), strides=(2, 2))(bn_conv2)
    Conv_block = convolutional_block(pooling_layer1, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    I_b1 = identity_block(Conv_block, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(I_b1, 3, [64, 64, 256], stage=2, block='c')
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    X = AveragePooling2D(pool_size=(2, 2),name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)
    model = Model(inputs = input_layer , outputs = X, name='ResNet50')
    return model

In [17]:
model = ResNet50(input_shape = (25, 25, 9,1), classes = 9)

In [18]:
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [20]:
history= model.fit(x=Xtrain,y=ytrain,batch_size=128,epochs=10)

Epoch 1/10
301/301 [==============================] - 35s 102ms/step - loss: 0.0360 - accuracy: 0.9897
Epoch 2/10
301/301 [==============================] - 31s 102ms/step - loss: 0.0174 - accuracy: 0.9950
Epoch 3/10
301/301 [==============================] - 30s 101ms/step - loss: 0.0138 - accuracy: 0.9958
Epoch 4/10
301/301 [==============================] - 31s 101ms/step - loss: 0.0121 - accuracy: 0.9961
Epoch 5/10
301/301 [==============================] - 31s 101ms/step - loss: 0.0188 - accuracy: 0.9944
Epoch 6/10
301/301 [==============================] - 31s 102ms/step - loss: 0.0039 - accuracy: 0.9986
Epoch 7/10
301/301 [==============================] - 30s 101ms/step - loss: 0.0059 - accuracy: 0.9986
Epoch 8/10
301/301 [==============================] - 31s 102ms/step - loss: 0.0058 - accuracy: 0.9984
Epoch 9/10
301/301 [==============================] - 30s 101ms/step - loss: 0.0142 - accuracy: 0.9964
Epoch 10/10
301/301 [==============================] - 31s 102ms/step - l

In [21]:
Xtest = Xtest.reshape(-1, 25, 25, 9,1)
Xtest.shape

(4278, 25, 25, 9, 1)

In [22]:
ytest = utils.to_categorical(ytest)
ytest.shape


(4278, 9)

In [23]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

134/134 [==============================] - 7s 23ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       648
           1       0.99      1.00      1.00      1871
           2       0.96      1.00      0.98       204
           3       1.00      1.00      1.00       315
           4       1.00      1.00      1.00       152
           5       1.00      0.96      0.98       482
           6       1.00      0.96      0.98       129
           7       0.98      0.99      0.99       384
           8       1.00      1.00      1.00        93

    accuracy                           0.99      4278
   macro avg       0.99      0.99      0.99      4278
weighted avg       0.99      0.99      0.99      4278

